# Quais os imóveis que House Rocket deveriam comprar?
- coletador os dados no kangle
- agrupar por região (zipcode)
- Dentro de cada região, vou fazer a mediana dos preços dos imóveis
- E entao sugerir a compra dos imóveis que estão abaixo do valor dessa mediana e com boas condições de compra.

- Exemlo de Tabela:
    Imóvel Cód Região\ Preço do Imóvel \ Preço da Mediana \ Condição\ Status:
    328545            \ 450 mil         \ 500 mil         \ boa     \Compra
    48578              \625 mil          \ 500 mil        \boa     \ Não Compra
    58789              \358 mil           \ 485 mil       \Não      \Não Compra 



In [1]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv('./kc_house_data.csv')

In [3]:
df

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,263000018,20140521T000000,360000.0,3,2.50,1530,1131,3.0,0,0,...,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
21609,6600060120,20150223T000000,400000.0,4,2.50,2310,5813,2.0,0,0,...,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
21610,1523300141,20140623T000000,402101.0,2,0.75,1020,1350,2.0,0,0,...,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
21611,291310100,20150116T000000,400000.0,3,2.50,1600,2388,2.0,0,0,...,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287


In [58]:
# conventando em datetime as datas
df['date'] = pd.to_datetime(df['date'])


# H1: Imóveis que possuem vista para água, são 30% mais caros, na média.
# H2: Imóveis com data de construção menor que 1955, são 50% mais baratos, na média.
# H3: Imóveis sem porão possuem sqrt_lot, são 50%
# maiores do que com porão.
# H4: O crescimento do preço dos imóveis YoY ( Year over Year ) é de 10%
# H5: Imóveis com 3 banheiros tem um crescimento MoM

In [61]:
# estastiticas das hipoteses:
#waterfront , zipcode, yr_built, sqft_lot,date, bathrooms, bedrooms, price, floors

df_waterfront = df[['waterfront','zipcode']].groupby('zipcode').median().reset_index()
df_price = df[['zipcode','price']].groupby('zipcode').median().reset_index()
df_yr_built= df[['zipcode','yr_built']].groupby('zipcode').median().reset_index()
df_sqft_lot = df[['zipcode','sqft_lot']].groupby('zipcode').median().reset_index()
df_bedrooms = df[['zipcode','bedrooms']].groupby('zipcode').median().reset_index()
df_bathrooms = df[['zipcode','bathrooms']].groupby('zipcode').median().reset_index()
df_floors = df[['zipcode','floors']].groupby('zipcode').median().reset_index()



In [67]:
#merge as tabelas criadas
m1 = pd.merge(df_waterfront,df_price, on='zipcode', how='inner')
m2 = pd.merge(m1,df_yr_built,on='zipcode',how='inner')
m3 = pd.merge(m2,df_sqft_lot,on='zipcode', how='inner')
m4 = pd.merge(m3, df_bedrooms, on= 'zipcode', how='inner')
m5 = pd.merge(m4, df_bathrooms, on='zipcode', how='inner')
m6 = pd.merge(m5, df_floors, on='zipcode', how='inner')

,zipcode,waterfront,price,yr_built,sqft_lot
0,98001,0.0,260000.0,1981.0,9287.0
1,98002,0.0,235000.0,1966.0,7282.0
2,98003,0.0,267475.0,1975.0,8816.0
3,98004,0.0,1150000.0,1965.0,11119.0
4,98005,0.0,765475.0,1967.0,12220.0
...,...,...,...,...,...
65,98177,0.0,554000.0,1956.0,8210.0
66,98178,0.0,278277.0,1952.0,7200.0
67,98188,0.0,264000.0,1959.5,8913.0
68,98198,0.0,265000.0,1962.0,8589.5
